In [ ]:
import os
import sys
print(os.environ['PYSPARK_PYTHON'])
print(os.environ['PYSPARK_DRIVER_PYTHON'])
# os.environ['PYSPARK_PYTHON'] = "/data/dataanalyst/anaconda3/envs/data_env/bin/python"
# os.environ['PYSPARK_DRIVER_PYTHON'] = "/data/dataanalyst/anaconda3/envs/data_env/bin/python"

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
os.environ['PYSPARK_DRIVER_PYTHON'] = "python"

In [ ]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--jars='/home/ml_data_analyst/.ivy2/jars/' pyspark-shell"
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.elasticsearch:elasticsearch-hadoop:7.1.0 pyspark-shell"

# https://spark.apache.org/docs/latest/api/python/user_guide/python_packaging.html
# config này khi muốn chạy các thư viện python trên executor mà k cần tải trên các executor
# (env_36) ml_data_analyst@hadoop532:~$ conda pack -f -o pyspark_conda_env_28_02_2023.tar.gz
# chú ý cái #environment match với cái env PYSPARK_PYTHON ở trên
os.environ['PYSPARK_SUBMIT_ARGS'] = "--archives /home/ml_data_analyst/pyspark_conda_env_28_02_2023.tar.gz#environment pyspark-shell"

# nếu tải jar rồi add thì chỉ cần tải 1 lần, tuy nhiên sẽ hạn chế nếu muốn đổi version hay jh đó
# pack dạng org:name:ver là chuẩn của ivy (có thể lấy trên maven)
# --packages thi no se tai ve tai 
# (env_36) ml_data_analyst@hadoop532:~$ ll /home/ml_data_analyst/.ivy2/jars/
# total 2980
# drwxrwxr-x 2 ml_data_analyst ml_data_analyst    4096 Feb 28 10:18 ./
# drwxrwxr-x 4 ml_data_analyst ml_data_analyst    4096 Feb 27 22:34 ../
# -rw-rw-r-- 1 ml_data_analyst ml_data_analyst 1011702 May 16  2019 org.elasticsearch_elasticsearch-hadoop-7.1.0.jar
# -rw-rw-r-- 1 ml_data_analyst ml_data_analyst 1011971 May 23  2019 org.elasticsearch_elasticsearch-hadoop-7.1.1.jar
# -rw-rw-r-- 1 ml_data_analyst ml_data_analyst 1012404 Dec 17  2019 org.elasticsearch_elasticsearch-hadoop-7.5.1.jar

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

import findspark
findspark.add_packages("org.elasticsearch.client:rest:5.1.2,org.elasticsearch:elasticsearch-spark-30_2.12:7.12.0,commons-httpclient:commons-httpclient:3.1")
findspark.init('/opt/spark320')
#org.apache.httpcomponents:httpclient:4.1.1,io.netty:netty-all:4.1.63.Final

conf = SparkConf().setAppName('long test es')  
conf.set('spark.yarn.queue', 'datamining')
conf.set('spark.kerberos.principal', 'mlbigdata/dm@HADOOP.SECURE')
conf.set('spark.kerberos.keytab', '/storage1/ml_data_analyst/mlbigdata.keytab')
conf.set('spark.driver.memory', '5g')
conf.set('spark.cores.max', '1')
conf.set('spark.executor.memory', '1G')
conf.set('spark.executor.instances', '5')
conf.set('spark.kerberos.access.hadoopFileSystems', 'hdfs://172.18.5.86:9000')
spark = SparkSession.builder.master("yarn").config(conf=conf).getOrCreate()

In [ ]:
myquery = """{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "domain": "sport5.vn"
          }
        }
      ]
    }
  }
}
"""

es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.query", myquery)  # filter trc load se nhanh hon
    .option("pushdown", "true")
    .option("es.read.field.include", "content,url,domain")
    .option("es.read.field.exclude", "")  # nếu có trường trong list bên trên thì bỏ qua
    .option("es.nodes","10.3.49.177:9200")
    .option("es.field.read.empty.as.null","no")
    .option("es.read.metadata",True)
    .option("es.net.http.auth.user","urldata")
    .option("es.net.http.auth.pass","3K75Fu1zijmanWP7OSl")
)

# .option('es.read.field.as.array.include','*')   # Caused by: java.lang.RuntimeException: scala.collection.convert.Wrappers$JListWrapper is not a valid external type for schema of bigint


In [ ]:
%%time
sysmon_df = es_reader.load("urldata_2022_04")
sysmon_df.select("_metadata").show(2, False)
id_doc = sysmon_df.select(sysmon_df._metadata._id)

In [ ]:
from elasticsearch import Elasticsearch

def get_es():
    from elasticsearch import Elasticsearch
    host = "10.1.11.1"
    port = "9200"
    username = "user"
    password = "pass"

    es = Elasticsearch(
            [f"{host}:{port}"],
            http_auth=(username, password) 
        )

        # Kiểm tra xem kết nối có thành công hay không
    if es.ping():
        print("Kết nối đến Elasticsearch cluster thành công.")
        return es
    else:
        print("Không thể kết nối đến Elasticsearch cluster.")
        return None

In [ ]:
def process_partition(iter_row):
    es = get_es()
    print(es.info())
    out = []
    for row in iter_row: 
        doc = es.get(index="urldata_2022_04", id=row['_metadata']['_id'])
        out.append((doc['_source']['timeCreate'],1))
        
    return outb

In [ ]:
result_rdd = sysmon_df.rdd.mapPartitions(process_partition).cache()
print(result_rdd.collect())

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType
deptSchema = StructType([       
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True)
])

deptDF1 = spark.createDataFrame(result_rdd, schema = deptSchema)
deptDF1.show()
